# Part-of-Speech tagging and Named Entity Recognition

In acest laborator vom vorbi despre mai multe tool-uri din modului [Spacy](), impreuna cu alte probleme actuale din NLP.



In continuare ne vom axa pe exemple pentru limba romana.

In [ ]:
! pip install -U spacy
! python -m spacy download ro_core_news_sm

In [ ]:
import spacy
from spacy.lang.ro.examples import sentences 
from tabulate import tabulate
import os

nlp = spacy.load("ro_core_news_sm")

## Analiza morfologica

POS (part of speech) este partea de vorbire la nivel macro.

 - ADJ: adjective, e.g. big, old, green, incomprehensible, first  
 - ADP: adposition, e.g. in, to, during  
 - ADV: adverb, e.g. very, tomorrow, down, where, there  
 - AUX: auxiliary, e.g. is, has (done), will (do), should (do)  
 - CONJ: conjunction, e.g. and, or, but  
 - CCONJ: coordinating conjunction, e.g. and, or, but  
 - DET: determiner, e.g. a, an, the  
 - INTJ: interjection, e.g. psst, ouch, bravo, hello  
 - NOUN: noun, e.g. girl, cat, tree, air, beauty  
 - NUM: numeral, e.g. 1, 2017, one, seventy-seven, IV, MMXIV  
 - PART: particle, e.g. ’s, not,  
 - PRON: pronoun, e.g I, you, he, she, myself, themselves, somebody  
 - PROPN: proper noun, e.g. Mary, John, London, NATO, HBO  
 - PUNCT: punctuation, e.g. ., (, ), ?  
 - SCONJ: subordinating conjunction, e.g. if, while, that  
 - SYM: symbol, e.g. $, %, §, ©, +, −, ×, ÷, =, :), 😝  
 - VERB: verb, e.g. run, runs, running, eat, ate, eating  
 - X: other, e.g. sfpksdpsxmsa  
 - SPACE: space, e.g.


Tag-ul este un pos fine grained, ce poate fi vazut ca o prescurtare a descrierii obtinute din morph.

In [ ]:
doc = nlp(sentences[0])

descriptions = []
for token in doc:
    desc = [token.text, token.pos_, token.tag_, token.morph]
    descriptions.append(desc)

print(tabulate(descriptions, headers=["word", "pos", "tag", "morph"]))

word        pos    tag      morph
----------  -----  -------  --------------------------------------------------------------
Apple       NOUN   Ncfp-n   Definite=Ind|Gender=Fem|Number=Plur
plănuiește  AUX    Vmip3s   Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin
să          PART   Qs       Mood=Sub
cumpere     AUX    Vmsp3    Mood=Sub|Person=3|Tense=Pres|VerbForm=Fin
o           DET    Tifsr    Case=Acc,Nom|Gender=Fem|Number=Sing|PronType=Ind
companie    NOUN   Ncfsrn   Case=Acc,Nom|Definite=Ind|Gender=Fem|Number=Sing
britanică   ADJ    Afpfsrn  Case=Acc,Nom|Definite=Ind|Degree=Pos|Gender=Fem|Number=Sing
pentru      ADP    Spsa     AdpType=Prep|Case=Acc
un          DET    Timsr    Case=Acc,Nom|Gender=Masc|Number=Sing|PronType=Ind
miliard     NUM    Mcms-ln  Definite=Ind|Gender=Masc|NumForm=Word|NumType=Card|Number=Sing
de          ADP    Spsa     AdpType=Prep|Case=Acc
dolari      NOUN   Ncmp-n   Definite=Ind|Gender=Masc|Number=Plur


Foarte util in detectia de mental health disorders. Conform unor studii psihologice oamenii diagnosticati cu depresie tind sa foloseasca mai mult pronumele "I", verbe la persoana intai si alte adverbe "nothing", "never" etc.. Mai multe detalii [aici](https://aclanthology.org/W18-4102.pdf).


## Dependency parsing

In [ ]:
from spacy import displacy

displacy.render(doc, style="dep", options={"distance": 100}, jupyter=True)

In [ ]:
# pentru a intelege relatiile
spacy.explain('ccomp')

'clausal complement'

In [ ]:
# https://spacy.io/usage/linguistic-features

dependency_features = []
for token in doc:
    dependency_features.append([token.text, token.dep_, token.head.text, token.head.pos_, [child for child in token.children]])
print(tabulate(dependency_features, headers=["word", "dependency", "parent", "pos", "children"]))

word        dependency    parent      pos    children
----------  ------------  ----------  -----  -----------------------
Apple       nsubj         plănuiește  AUX    []
plănuiește  ROOT          plănuiește  AUX    [Apple, cumpere]
să          mark          cumpere     AUX    []
cumpere     ccomp         plănuiește  AUX    [să, companie, miliard]
o           det           companie    NOUN   []
companie    obj           cumpere     AUX    [o, britanică]
britanică   amod          companie    NOUN   []
pentru      case          miliard     NUM    []
un          det           miliard     NUM    []
miliard     obl           cumpere     AUX    [pentru, un, dolari]
de          case          dolari      NOUN   []
dolari      nmod          miliard     NUM    [de]


## Lematizare folosind POS

Am vazut in al doilea laborator cum putem folosi atat `nltk` cat si `spacy` pentru a extrage lemele (formele de dictionar) ale unor cuvinte.

In general, lematizarea din `nltk` nu foloseste informatii despre partea de vorbire, pe cand in `spacy` in pipelineul de preprocesare sunt detectate partile de vorbire si folosite in lematizare. Putem insa sa ne folosim de POS si in cazul `nltk` astfel:

In [ ]:
import nltk
nltk.download('wordnet')
wl = nltk.WordNetLemmatizer()  # instantiem un lemmatizer

In [ ]:
print(wl.lemmatize("being"))
print(wl.lemmatize("being", pos="n"))
print(wl.lemmatize("being", pos="v"))

being
being
be


In [ ]:
print(wl.lemmatize("painted"))
print(wl.lemmatize("painted", pos="v"))
print(wl.lemmatize("painted", pos="a"))

painted
paint
painted


POS in `nltk` sunt:
- `n` (noun/substantiv)
- `v` (verb)
- `a` (adjectiv)
- `r` (adverb).

# Name Entity Recognition (NER)

Un task important de recunoastere a numelor proprii din texte. Fiecare corpus din spacy are o lista de label-uri pe care le putem vizualiza prin apelul de mai jos. 

**ORG** - companii, agentii, etc. (mai frecvent in corpusul de engleza)  
**GPE** - entitati geopolitice precum tari, orase, state, etc.



In [ ]:
nlp.get_pipe("ner").labels

('DATETIME',
 'EVENT',
 'FACILITY',
 'GPE',
 'LANGUAGE',
 'LOC',
 'MONEY',
 'NAT_REL_POL',
 'NUMERIC_VALUE',
 'ORDINAL',
 'ORGANIZATION',
 'PERIOD',
 'PERSON',
 'PRODUCT',
 'QUANTITY',
 'WORK_OF_ART')

Mai jos putem vedea pipeline-ul aplicat pe o propozitie concreta in romana. Rezultatul este o lista de span-uri, fiecare span continand mai multe token-uri. Fiecare token are flag-ul IOB cu urmatoarea semnificatie:

I – Token is inside an entity.  
O – Token is outside an entity.  
B – Token is the beginning of an entity.  

In [ ]:
print(sentences[2], ":\n")
doc = nlp(sentences[2])
entities = []
for ent in doc.ents:
    ent_description = []
    for token in ent:
        ent_description.append(f'{token.text} - {token.ent_iob_}')
    
    entities.append([ent.text, (ent.start_char, ent.end_char), ent.label_, '; '.join(ent_description)])
print(tabulate(entities, headers=["word", "position", "label", "token - IOB"]))

Londra este un oraș mare în Regatul Unit :

word          position    label    token - IOB
------------  ----------  -------  ---------------------
Londra        (0, 6)      GPE      Londra - B
Regatul Unit  (28, 40)    GPE      Regatul - B; Unit - I


In versiunea 2 de spacy, modelul NER este o retea convolutionala (laborator 4), pe cand in versiunea 3, avem de-a face cu un [transformer](https://arxiv.org/pdf/1706.03762.pdf).  

In [ ]:
def get_label(layer):
    layer_name = layer.name
    nO = layer.get_dim("nO") if layer.has_dim("nO") else "?"
    nI = layer.get_dim("nI") if layer.has_dim("nI") else "?"
    return f"{layer.name}|({nO}, {nI})".replace('>>', '\n')

ner = nlp.get_pipe('ner')
for i, ner_layer in enumerate(ner.model.layers):
    print(f"Layer no {i}:")
    print(get_label(ner_layer))
    print()

Layer no 0:
extract_features
list2ragged
with_array(ints-getitem
hashembed|ints-getitem
hashembed|ints-getitem
hashembed|ints-getitem
hashembed|ints-getitem
hashembed)
with_array(maxout
layernorm
dropout)
ragged2list
with_array(residual(expand_window
maxout
layernorm
dropout)
residual(expand_window
maxout
layernorm
dropout)
residual(expand_window
maxout
layernorm
dropout)
residual(expand_window
maxout
layernorm
dropout))
list2array
linear|(64, ?)

Layer no 1:
precomputable_affine|(64, 64)

Layer no 2:
linear|(66, 64)



In [ ]:
!wget https://raw.githubusercontent.com/dumitrescustefan/ronec/master/data/train.json

--2022-04-05 20:12:30--  https://raw.githubusercontent.com/dumitrescustefan/ronec/master/data/train.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10753146 (10M) [text/plain]
Saving to: ‘train.json’

train.json          100%[===================>]  10.25M  25.8MB/s    in 0.4s    

2022-04-05 20:12:31 (25.8 MB/s) - ‘train.json’ saved [10753146/10753146]



In [ ]:
import json

with open('train.json', 'r') as fin:
    ronec = json.load(fin)

In [ ]:
print(ronec[1].keys())
print(ronec[1]["id"])
print(ronec[1]["tokens"])
print(ronec[1]["ner_tags"])
print(ronec[1]["ner_ids"])
print(ronec[1]["space_after"])

dict_keys(['id', 'ner_tags', 'ner_ids', 'tokens', 'space_after'])
11865
['Vechiul', 'oraș', 'Visoki', 'a', 'fost', 'un', 'faimos', 'castel', 'regal', 'medieval', 'în', 'timpul', 'secolului', 'al', 'XIV', '-lea', ',', 'situat', 'în', 'Visoko', ',', 'Bosnia', 'și', 'Herțegovina', '.']
['O', 'O', 'B-GPE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-DATETIME', 'I-DATETIME', 'I-DATETIME', 'I-DATETIME', 'O', 'O', 'O', 'B-GPE', 'O', 'B-GPE', 'I-GPE', 'I-GPE', 'O']
[0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 17, 18, 18, 18, 0, 0, 0, 5, 0, 5, 6, 6, 0]
[True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False, True, True, True, True, True, True, True, False, False]


In [ ]:
import re

dataset_max_len = 100  # extragem doar primele 100 de exemple din ronec
dataset = []
for i, entry in enumerate(ronec):
    if i > 100:
        break

    entities = []
    full_text = ''
    is_outside = True
    for i, (token, ner_tag, is_space) in enumerate(zip(entry['tokens'], entry['ner_tags'], entry['space_after'])):
        current_space = ' ' if is_space else ''

        if ner_tag.startswith('B-'):
            is_outside = False
            entity = ''
            entity_start = len(full_text)
            entity_name = ner_tag.split('-')[-1]

        elif ner_tag == 'O' and not is_outside:
            is_outside = True
            entity_end = len(full_text)
            if entity[-1] == ' ':
                entity_end -= 1
                entity = entity[:-1]
            # print(entity_start, entity_end, entity_name, entity)
            entities.append((entity_start, entity_end, entity_name))

        if not is_outside:
           entity += token + current_space
        full_text += token + current_space
    
    dataset.append((full_text, {
        "entities": entities
    }))


In [ ]:
print(dataset[1])

('Vechiul oraș Visoki a fost un faimos castel regal medieval în timpul secolului al XIV-lea, situat în Visoko , Bosnia și Herțegovina.', {'entities': [(13, 19, 'GPE'), (69, 89, 'DATETIME'), (101, 107, 'GPE'), (110, 131, 'GPE')]})


In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

train_dataset, test_dataset, _, _ = train_test_split(dataset, np.zeros(len(dataset)), test_size=0.1, random_state=101)

Avand datele impartite pe antrenare si testare, pentru a antrena un nou model NER, avem nevoie de un alt NLP pipeline, caruia ii adaugam o componenta (pipe) de NER, dupa cum urmeaza:

In [ ]:
nlp = spacy.blank('ro')  

if 'ner' not in nlp.pipe_names:
    ner = nlp.add_pipe('ner', last=True)
else:
    ner = nlp.get_pipe('ner')

Adaugam label-urile din dataset-ul nostru, respectand formatul initial.

In [ ]:
for _, annotations in dataset:
    for ent in annotations["entities"]:
        ner.add_label(ent[2])

In [ ]:
import random
from tqdm import tqdm
from spacy.training.example import Example
n_epochs = 100

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # Antrenam doar NER-ul     
    optimizer = nlp.begin_training()
    for epoch in range(n_epochs):
        random.shuffle(train_dataset)
        losses = {}
        for text, annotations in tqdm(train_dataset):
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)
            nlp.update(
                [example],  
                drop=0.35,  
                sgd=optimizer,
                losses=losses
            )
        print(losses)

 26%|█████████████                                      | 23/90 [00:00<00:00, 71.19it/s]/home/bogdan/work/FMI/lab_NLP_an3/.env/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "Dar și producătorul casetei audio intitulată 'Talk..." with entities "[(45, 78, 'WORK_OF_ART'), (80, 84, 'DATETIME'), (1...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
 42%|█████████████████████▌                             | 38/90 [00:00<00:00, 55.25it/s]/home/bogdan/work/FMI/lab_NLP_an3/.env/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "Sparta a luat insula de la Argos la începutul seco..." with entities "[(0, 6, 'GPE'), (27, 32, 'GPE'), (36, 65, 'DATETIM...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), e

{'ner': 1181.5347758118269}


100%|███████████████████████████████████████████████████| 90/90 [00:01<00:00, 61.24it/s]


{'ner': 801.6053013760431}


100%|███████████████████████████████████████████████████| 90/90 [00:01<00:00, 62.70it/s]


...

{'ner': 44.73341953390249}


100%|███████████████████████████████████████████████████| 90/90 [00:01<00:00, 61.75it/s]


{'ner': 41.845683411937436}


100%|███████████████████████████████████████████████████| 90/90 [00:01<00:00, 61.02it/s]

{'ner': 29.4656435611218}


Salvam modelul intr-un folder prestabilit, apoi il testam folosind un scorrer din spacyv3. Daca vreti sa folositi spacy2, se procedeaza altfel, precum am citit [aici](https://stackoverflow.com/questions/44827930/evaluation-in-a-spacy-ner-model).

In [ ]:
model_folder = 'trained_ner'
if not os.path.exists(model_folder):
    os.makedirs(model_folder)
nlp.to_disk(model_folder)

In [ ]:
from spacy.scorer import Scorer
scorer = Scorer()

comparing_examples = []
for text, anotations in test_dataset:     
    pred_value = nlp(text)
    example = Example.from_dict(pred_value, anotations)
    comparing_examples.append(example)
    
result = scorer.score(comparing_examples)

print(f"Modelul s-a terminat cu F1 = {result['ents_f']}, precizia = {result['ents_p']} si recall-ul = {result['ents_r']} \nIar pentru fiecare clasa:")
print(json.dumps(result['ents_per_type'], indent=4))

Modelul s-a terminat cu F1 = 0.47457627118644063, precizia = 0.509090909090909 si recall-ul = 0.4444444444444444 
Iar pentru fiecare clasa:
{
    "GPE": {
        "p": 0.5833333333333334,
        "r": 0.5833333333333334,
        "f": 0.5833333333333334
    },
    "DATETIME": {
        "p": 0.6363636363636364,
        "r": 0.6363636363636364,
        "f": 0.6363636363636364
    },
    "MONEY": {
        "p": 0.0,
        "r": 0.0,
        "f": 0.0
    },
    "PERSON": {
        "p": 0.4782608695652174,
        "r": 0.39285714285714285,
        "f": 0.4313725490196078
    },
    "NAT_REL_POL": {
        "p": 0.5,
        "r": 0.25,
        "f": 0.3333333333333333
    },
    "ORG": {
        "p": 0.0,
        "r": 0.0,
        "f": 0.0
    },
    "ORDINAL": {
        "p": 0.0,
        "r": 0.0,
        "f": 0.0
    },
    "WORK_OF_ART": {
        "p": 0.0,
        "r": 0.0,
        "f": 0.0
    },
    "LOC": {
        "p": 0.0,
        "r": 0.0,
        "f": 0.0
    },
    "PERIOD": {
   

/home/bogdan/work/FMI/lab_NLP_an3/.env/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "Anglia și Tunisia s-au înfruntat la Cupa Mondială ..." with entities "[(0, 6, 'GPE'), (10, 17, 'GPE'), (50, 58, 'DATETIM...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/home/bogdan/work/FMI/lab_NLP_an3/.env/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "Adi Petre a marcat din nou pentru Esbjerg: statist..." with entities "[(0, 9, 'PERSON'), (34, 41, 'ORG'), (62, 69, 'PERS...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/home/bogdan/work/FMI/lab_NLP_an3/.env/lib/python3.8/site-packages

Alte metrici prin care putem evalua si interpreta un model NER, putem gasi [aici](https://towardsdatascience.com/a-pathbreaking-evaluation-technique-for-named-entity-recognition-ner-93da4406930c).

# TASK
## Deadline: 21 aprilie ora 23:59.

Formular pentru trimiterea temei: https://forms.gle/F3hnh16W8tfuuoXQ6

1. Descarcati un text de ~20 de fraze in limba engleza (de exemplu primele 20 de fraze dintr-un articol wikipedia).
    - folosind spacy POS-tagging, extrageti separat toate substantivele, toate verbele, toate adjectivele si toate adverbele (afisati cate cuvinte din fiecare tip ati gasit); puteti gasi [aici](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html) o lista cu tagurile pentru engleza.
    - pentru aceste cuvinte extrase, determinati lema folosind nltk wordnet lemmatizer; pentru fiecare cuvant determinati lema atat fara a specifica tipul de parte de vorbire, cat si specificandu-l; pentru ce procent din cuvinte rezultatele pentru cele doua variante sunt diferite.
2. Selectati din RONEC un subset mai mare de date si antrenati un model de NER (folosind spacy, ca in laborator). Pentru fiecare clasa afisati precision, recall, f1.
    - Care este clasa cu scorul cel mai bun? Dar cea cu scorul cel mai slab?

In [ ]:
! pip install wikipedia

In [ ]:
! pip install -U spacy
! python -m spacy download en_core_web_sm

In [3]:
import spacy
from spacy.lang.ro.examples import sentences 
from tabulate import tabulate
import os

nlp = spacy.load("en_core_web_sm")

In [4]:
import wikipedia
page_title = "Jazz"
wikipedia.set_lang("en")
page = wikipedia.page(page_title, auto_suggest=False)

propozitii = page.content.split('.')[:20]

In [5]:
substantive = set()
verbe = set()
adjective = set()
adverbe = set()
cuvinte = {
    'NN':substantive,
    'VB':verbe,
    'JJ':adjective,
    'RB':adverbe
  }
for propozitie in propozitii:
  doc = nlp(propozitie)
  for token in doc:
    tag = token.tag_[:2]
    if tag in cuvinte.keys():
      cuvinte[tag].add(token.text)
    

In [ ]:
list(substantive)[:10]

['=',
 'call',
 'definition',
 'United',
 'pitch',
 'jas',
 'National',
 'sound',
 'Kansas',
 'band']

In [11]:
import nltk
nltk.download('wordnet')
wl = nltk.WordNetLemmatizer() 
same_lemma = 0
total_words_len = len(substantive) + len(verbe) + len(adjective) + len(adverbe)
translate= {
    'NN':'n',
    'VB':'v',
    'JJ':'a',
    'RB':'r'
}
for k,v in cuvinte.items():
  for cuvant in v:
    fara_specificare = wl.lemmatize(cuvant)
    cu_specificare = wl.lemmatize(cuvant,pos=translate[k])
    if(fara_specificare == cu_specificare):
      same_lemma += 1
  
print(str(same_lemma/total_words_len*100)+"%")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
78.77551020408163%


In [12]:
!wget https://raw.githubusercontent.com/dumitrescustefan/ronec/master/data/train.json

--2022-04-07 17:07:21--  https://raw.githubusercontent.com/dumitrescustefan/ronec/master/data/train.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10753146 (10M) [text/plain]
Saving to: ‘train.json’

train.json          100%[===================>]  10.25M  --.-KB/s    in 0.08s   

2022-04-07 17:07:21 (128 MB/s) - ‘train.json’ saved [10753146/10753146]



In [14]:
import json

with open('train.json', 'r') as fin:
    ronec = json.load(fin)

In [23]:

import re

dataset_max_len = 2000
dataset = []
for i, entry in enumerate(ronec):
    if i > 2000:
        break

    entities = []
    full_text = ''
    is_outside = True
    for i, (token, ner_tag, is_space) in enumerate(zip(entry['tokens'], entry['ner_tags'], entry['space_after'])):
        current_space = ' ' if is_space else ''

        if ner_tag.startswith('B-'):
            is_outside = False
            entity = ''
            entity_start = len(full_text)
            entity_name = ner_tag.split('-')[-1]

        elif ner_tag == 'O' and not is_outside:
            is_outside = True
            entity_end = len(full_text)
            if entity[-1] == ' ':
                entity_end -= 1
                entity = entity[:-1]
            # print(entity_start, entity_end, entity_name, entity)
            entities.append((entity_start, entity_end, entity_name))

        if not is_outside:
           entity += token + current_space
        full_text += token + current_space
    
    dataset.append((full_text, {
        "entities": entities
    }))


In [24]:
print(len(dataset))

2001


In [25]:
from sklearn.model_selection import train_test_split
import numpy as np

train_dataset, test_dataset, _, _ = train_test_split(dataset, np.zeros(len(dataset)), test_size=0.1, random_state=101)

In [26]:
nlp = spacy.blank('ro')  

if 'ner' not in nlp.pipe_names:
    ner = nlp.add_pipe('ner', last=True)
else:
    ner = nlp.get_pipe('ner')

In [28]:
import random
from tqdm import tqdm
from spacy.training.example import Example
n_epochs = 30

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):   
    optimizer = nlp.begin_training()
    for epoch in range(n_epochs):
        random.shuffle(train_dataset)
        losses = {}
        for text, annotations in tqdm(train_dataset):
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)
            nlp.update(
                [example],  
                drop=0.35,  
                sgd=optimizer,
                losses=losses
            )
        print(losses)

  0%|          | 0/1800 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "Astfel, populația cu un consum de până la 2400 de ..." with entities "[(8, 17, 'PERSON'), (42, 60, 'QUANTITY'), (76, 82,...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
  1%|          | 19/1800 [00:00<01:11, 24.94it/s]/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "Ofertele pentru concesionarea celor 22 de parcele ..." with entities "[(36, 38, 'NUMERIC'), (79, 82, 'QUANTITY'), (83, 9...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.


{'ner': 18313.8798930568}


100%|██████████| 1800/1800 [01:21<00:00, 22.14it/s]


{'ner': 13599.549737670526}


100%|██████████| 1800/1800 [01:21<00:00, 22.11it/s]


...

{'ner': 5001.0139719570025}


100%|██████████| 1800/1800 [01:23<00:00, 21.50it/s]


{'ner': 4937.865795396152}


100%|██████████| 1800/1800 [01:23<00:00, 21.68it/s]

{'ner': 4708.963659468097}


In [29]:
model_folder = 'trained_ner'
if not os.path.exists(model_folder):
    os.makedirs(model_folder)
nlp.to_disk(model_folder)

In [30]:
from spacy.scorer import Scorer
scorer = Scorer()

comparing_examples = []
for text, anotations in test_dataset:     
    pred_value = nlp(text)
    example = Example.from_dict(pred_value, anotations)
    comparing_examples.append(example)
    
result = scorer.score(comparing_examples)

print(f"Modelul s-a terminat cu F1 = {result['ents_f']}, precizia = {result['ents_p']} si recall-ul = {result['ents_r']} \nIar pentru fiecare clasa:")
print(json.dumps(result['ents_per_type'], indent=4))

/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "26 -Directiva Consiliului din 25 noiembrie 2003 (J..." with entities "[(0, 2, 'NUMERIC'), (30, 47, 'DATETIME'), (54, 56,...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "După severul diagnostic, mama Antoniei revine în I..." with entities "[(30, 38, 'PERSON'), (49, 55, 'GPE'), (108, 115, '...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/usr/local/lib/python3

Modelul s-a terminat cu F1 = 0.6685902720527617, precizia = 0.6652994257588187 si recall-ul = 0.671913835956918 
Iar pentru fiecare clasa:
{
    "DATETIME": {
        "p": 0.8571428571428571,
        "r": 0.8571428571428571,
        "f": 0.8571428571428571
    },
    "PERSON": {
        "p": 0.6211764705882353,
        "r": 0.6423357664233577,
        "f": 0.6315789473684211
    },
    "NAT_REL_POL": {
        "p": 0.6075949367088608,
        "r": 0.6486486486486487,
        "f": 0.6274509803921569
    },
    "GPE": {
        "p": 0.7096774193548387,
        "r": 0.7810650887573964,
        "f": 0.7436619718309859
    },
    "ORG": {
        "p": 0.4642857142857143,
        "r": 0.5652173913043478,
        "f": 0.5098039215686274
    },
    "WORK_OF_ART": {
        "p": 0.42857142857142855,
        "r": 0.10344827586206896,
        "f": 0.16666666666666663
    },
    "NUMERIC": {
        "p": 0.8617021276595744,
        "r": 0.7864077669902912,
        "f": 0.8223350253807107
    },
  